In [ ]:
import pandas as pd
import numpy as np

import eikon as ek
import os
import sys
import datetime as dt

from dotenv import load_dotenv
from tqdm import tqdm
from bs4 import BeautifulSoup

sys.path.insert(0, r'c:\Users\joneh\master_thesis\src')
from main_utils import *
from db_utils import *

load_dotenv()
ek.set_app_key(os.getenv('EIKON_API_KEY'))


In [ ]:
start_date = dt.datetime(2021, 1, 1)
end_date = dt.datetime(2021, 12, 31)

ek.get_news_headlines('R:AMZN.O', date_from=start_date, date_to=end_date, count=100)

In [ ]:
import refinitiv.data as rd
rd.open_session()

In [ ]:
query = "Topic:CRU AND language:LEN"

start_date = dt.datetime(2023, 1, 1)
end_date = dt.datetime(2023, 12, 31)
dates = pd.date_range(start_date, end_date, freq='D')

df_list = []

for date in tqdm(dates):
    response = rd.content.news.headlines.Definition(
        query="Topic:CRU AND language:LEN", 
        extended_params={"searchIn": "FullStory"}, 
        count=1000, date_from=date, date_to=date
    ).get_data()
    df_list.append(response.data.df)

df = pd.concat(df_list)
display(df)

In [ ]:
# Enter filename here:
file_name = f'CRU_2023.csv'
# Enter relative path for saving the file:
relative_path = 'data/news'

df.to_csv(save_path(relative_path, file_name), index=True)

In [ ]:
start_date = dt.datetime(2023, 12, 22)
end_date = dt.datetime(2023, 12, 22)

ek.get_news_headlines('Topic:CRU AND Language:LEN', date_to=start_date, date_from=end_date, count=100)



In [ ]:
db_df = news_db_load('TRnews.db', 'news_filtered', dt_index=False)

In [ ]:

article_text = {}

for id in tqdm(db_df['storyId']):
    story = ek.get_news_story(id)
    soup = BeautifulSoup(story, 'html.parser')
    article_text[id] = soup.get_text()

db_df['article_text'] = db_df['storyId'].map(article_text)

In [ ]:
db_df['article_text'] = db_df['storyId'].map(article_text)

display(db_df.head(20))


In [ ]:
db_df